In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import swifter

import cleantext

pd.options.display.max_colwidth = 1000

In [2]:
max_previous = None

In [6]:
if max_previous is None:
    OUT = Path('~/data/ynacc_proc/replicate/threads')
else:
    OUT = Path('~/data/ynacc_proc/replicate/threads_prev' + str(max_previous))

BASE_PATH = Path('/mnt/data/datasets/ydata-ynacc-v1_0')
ANN1 = BASE_PATH/'ydata-ynacc-v1_0_expert_annotations.tsv'
ANN2 = BASE_PATH/'ydata-ynacc-v1_0_turk_annotations.tsv'
UNL = BASE_PATH/'ydata-ynacc-v1_0_unlabeled_conversations.tsv'
TRAIN_IDS = BASE_PATH/'ydata-ynacc-v1_0_train-ids.txt'

In [7]:
trainids = pd.read_csv(TRAIN_IDS, header=None)
df_an1 = pd.read_table(ANN1)
df_an1 = df_an1[df_an1['sdid'].isin(list(trainids[0]))]
df_an1 = df_an1[['sdid', 'text', 'commentindex']]
df_an1 = df_an1.drop_duplicates()
df_an1

,sdid,text,commentindex
0,53971,"These things happen , Every job has its dangers.",2
1,53971,"Sad to hear such a bad thing. Very dangerous job working on electricity. One questions though, why did they use a picture the Bates house from Psycho, on a Disney story? Or is that what the Paris Haunted Mansion/Phantom Manor looks like?",0
2,53971,Yes..because too many houses in EU look like the original Disney Hunted House so it didn't look scary enough. Bates Motel looks more American and that notion alone scares everyone.,1
3,135929,"I am frankly quite SICK of the phrase ""shoved down our throat"" You know what? Back in the newspaper and three network days you could say that...Now with 300 or more TV channels and an endless internet...You can keep your throat relatively clear of things you don't want...All you have to do is change the channel or click a link to something you DO like... So let's stop it with the ""shoved down our throat"" rhetoric.",0
4,135929,"Ya, I always wonder why the conservatives are on Yahoo!, whining about all the liberals, when they could be hanging out with their own kind, patting each other on the back, over at Faux News. Don't like what you see, switch the channel. No, they're a bit hypocritical about shoving things down peoples throats, and want to force they're way upon others.",1
5,135929,Great comment!,2
6,127970,"The result is a system where being poor becomes a de facto crime. Except, if he had NOT stolen a guitar, then he would not have HAD to post bail to begin with. He was not in jail for ""being poor""....he was there for theft, and rightly so. Next, liberals will say that ""stealing is not a capital crime""...but he was not ""executed"" for stealing. In fact, he had not even been convicted. He was killed by a couple of other thugs - probably for ""dissing"" them. Yes, this is a tragic situation. No, he should not be dead. However, he is solely responsible for being in the position where he was in jail to begin with. Jails are not happy places - and they are not supposed to be. They are supposed to be places that you avoid going to.",0
7,127970,They are also places where you are supposed not to be beaten to death either. If I locked up several animals all in one cage and came back some time later to find two of them had mauled and killed one on them I would be held accountable for animal cruelty at the least. I bet no one running this particular zoo ever is held accountable for this death tho.,2
8,127970,"Stop trying to make sense, it only confuses people, especially the stupid ones...and liberals.",1
9,127970,"Madd...sorry - I can't help it. Grim....are you calling criminals ""animals""??? Raaaaacist!!!! (jk)",3


In [8]:
df_an2 = pd.read_table(ANN2)
df_an2 = df_an2[df_an2['sdid'].isin(list(trainids[0]))]
df_an2 = df_an2[['sdid', 'text', 'commentindex']]
df_an2 = df_an2.drop_duplicates()
df_an2

,sdid,text,commentindex
21,45408,"Anyone that believes Hiliary Clinton can get information, especially info regarding potentially above top secret national security is pretty naive. Especially, with all the problems at the State Dept. I think Bill was turned away when he went barking up that tree. Anyway, the real losers here are the American public because we have no idea what goes on in these secret instillations and everything is black budgeted. I have never seen a UFO, alien, or any other mysterious thing. But, many others have and possibly there are these things out there? If so, would be nice to know the truth.",0
22,45408,"\""Anyone that believes Hiliary Clinton can get information, especially info regarding potentially above top secret national security is pretty naive.\"" Can't she just have someone email it to her private server?",1
23,45408,She probably has it on her private server.,2
24,45408,Hillary is a visionary,3
25,45408,"Don't know about her, but AMY Hall Dead Files is your best bet..",4
36,29371,"This major Jack #$%$ of a president is trying so hard to do anything he can to further the \""Racist\"" agenda. Once a \""Community Organizer\"" always a community Organizer!! He got his #$%$ handed to him in Germany Today as 1000's showed up to boo him and to let him know that Germany does not need him or his rhetoric. They showed strong ugly opposition to his TTP (Trans Pacific Partnership) and basically booed him and let him know that they weren't falling for his NWO plans. The world dislikes this traitor and see's right through him and all his \""Transparency\"". The thought of Obuma being booed off stage in a foreign country brought a big smile to my face. Wish someone had thrown a \""shoe\"" at him and really insulted him..like George Bush Got.",0
37,29371,You have major bigotry issues and I am white. What is your problem.,1
38,29371,Well since you have blacks in your family and you're white that means what?smh,2
39,29371,"I am white, conservative and I dont see this as a race issue, Tubman was an American hero. But if it makes you feel better there is word that Obama and Clinton are negotiating to be put on the $3 bill-it could be like the black and white cookie; racial harmony.",3
40,29371,"You know she was a \""hero\"" as a 100% definitive fact. Or is that just what some Biased book or biased media reported. Do you know the real \""Facts\"" ? I think NOT !!!!",4


In [9]:
df_notan = pd.read_csv(UNL, engine='python', sep='\t', quoting=3, error_bad_lines=False)
df_notan = df_notan[['sdid', 'text', 'commentindex']]

In [7]:
# not needed anmoyre
# df['text'] = df.apply(lambda x: 'xx_root_comment ' + x['text'] if pd.isnull(x['parentid']) else x['text'], axis=1)
# df['parentid'] = df.apply(lambda x: x['commentid'] if pd.isnull(x['parentid']) else x['parentid'], axis=1)

In [10]:
df = pd.concat([df_an1, df_an2, df_notan])
# clean up
df = df.dropna(subset=['text'])
df["commentindex"] = pd.to_numeric(df["commentindex"])
df

,sdid,text,commentindex
0,53971,"These things happen , Every job has its dangers.",2
1,53971,"Sad to hear such a bad thing. Very dangerous job working on electricity. One questions though, why did they use a picture the Bates house from Psycho, on a Disney story? Or is that what the Paris Haunted Mansion/Phantom Manor looks like?",0
2,53971,Yes..because too many houses in EU look like the original Disney Hunted House so it didn't look scary enough. Bates Motel looks more American and that notion alone scares everyone.,1
3,135929,"I am frankly quite SICK of the phrase ""shoved down our throat"" You know what? Back in the newspaper and three network days you could say that...Now with 300 or more TV channels and an endless internet...You can keep your throat relatively clear of things you don't want...All you have to do is change the channel or click a link to something you DO like... So let's stop it with the ""shoved down our throat"" rhetoric.",0
4,135929,"Ya, I always wonder why the conservatives are on Yahoo!, whining about all the liberals, when they could be hanging out with their own kind, patting each other on the back, over at Faux News. Don't like what you see, switch the channel. No, they're a bit hypocritical about shoving things down peoples throats, and want to force they're way upon others.",1
5,135929,Great comment!,2
6,127970,"The result is a system where being poor becomes a de facto crime. Except, if he had NOT stolen a guitar, then he would not have HAD to post bail to begin with. He was not in jail for ""being poor""....he was there for theft, and rightly so. Next, liberals will say that ""stealing is not a capital crime""...but he was not ""executed"" for stealing. In fact, he had not even been convicted. He was killed by a couple of other thugs - probably for ""dissing"" them. Yes, this is a tragic situation. No, he should not be dead. However, he is solely responsible for being in the position where he was in jail to begin with. Jails are not happy places - and they are not supposed to be. They are supposed to be places that you avoid going to.",0
7,127970,They are also places where you are supposed not to be beaten to death either. If I locked up several animals all in one cage and came back some time later to find two of them had mauled and killed one on them I would be held accountable for animal cruelty at the least. I bet no one running this particular zoo ever is held accountable for this death tho.,2
8,127970,"Stop trying to make sense, it only confuses people, especially the stupid ones...and liberals.",1
9,127970,"Madd...sorry - I can't help it. Grim....are you calling criminals ""animals""??? Raaaaacist!!!! (jk)",3


In [11]:
df['text'] = df['text'].swifter.apply(lambda x: cleantext.clean(x, lower=False, no_urls=True, no_emails=True, zero_digits=True))

Pandas Apply: 100%|██████████| 238512/238512 [01:29<00:00, 2668.77it/s]


In [12]:
df = df.drop_duplicates()

In [13]:
# get list of all comennts per thread
res = df.sort_values(by=['commentindex']).groupby('sdid').agg({'text': lambda x: list(x)}).reset_index()

In [14]:
res

,sdid,text
0,27,"[She was 00, not 00. And these prosecuting lawyers act like they have never done anything wrong or questionable in their life. If they look happy, and say they are happy, just leave them alone. Why ruin someone else's life, when there really isn't any good reason to do so?, Yea, that totally makes it alright for a teacher to bang one of his students. Herp derp., Yeah, because #$%$ feelings and human nature, that is all deranged., are you serious no its not ok to bang a student if shes in highschool and your her teacher no matter what her age is, Oblivion, you are not the brightest bulb in the room. A) The incident occurred in 0000 which would make the student 00 or 00 at the time of the incident not 00 and the teacher 00 or 00. B) It is and should be against the law for high school teachers to sleep with their students. And C) Any parent, which you obviously aren't, would be outraged that a teacher they are paying with their tax dollars was sleeping with his student, their daughter..."
1,44,"[There is no problem with an older man marrying an adult woman no matter how wide apart they are in years. Think about that same distance in age when the girl is just 00 and the man is 00. Seems like he is a cradle robber but not really. My wife before she was had known each other almost our entire lives as children before eventually marrying in 0000 when she was 00 and I was 00. We wanted to just live together and forget marriage but her family insisted on this big elaborate wedding we both hated. We are 0 years and 0 months apart in age with me being the oldest. We were together so long everyone considered us married already. Now just passing 00 years married it was the perfect match., Women tend to live 0 to 00 years longer than men. Therefore men should usually marry women that are that much older than them., Big difference. Your case the age difference is less than 0 yrs and I assume there wasn't any court case hanging over your head Their age difference is almost 00 yrs and s..."
2,80,"[I guess the law considers their case as being Cox blocked., I see what you did there. Nice., VERY clever!, :) Funny pun!]"
3,219,"[You could argue the case that at the time she was a minor and in school, and require her testimony on those grounds. Plus, if there are pictures of them during the time indicated, her testimony may not be needed, as those pics and texts are dated and should be enough to convict him., She could just refuse to say anything. They can't force her to talk. What are they going to do, jail her, the victim? These two seem happy, can't the prosecutors go find some real crime to solve? Maybe it's time to slash law enforcement budget and cut my taxes too., Maybe your just into humping underage kids, Michael Jackson., @USERNAME - good call. We should do the same for all rape victims; punish them until they take the stand against the accused. [endTrolling], No, we should eleminate the stupid laws that protect predators like this. It leads to abusive marriages where the spouse literally has no legal rights in court to protect themselves.]"
4,288,"[To the American mind, a child is a magical, innocent fairy creature incapable of knowing right from wrong or having any sort of physical urges common place to other human beings. They simply cannot and do not see children as simply being merely juvenile humans. -- J.R.R. Tolkien (Yes, the guy who wrote the Hobbits and Lord of the Rings - wrote some fascinating observances of this.)., ""To the American mind, a child is a magical, innocent fairy creature incapable of knowing right from wrong or having any sort of physical urges common place to other human beings. They simply cannot and do not see children as simply being merely juvenile humans."" -- J.R.R. Tolkien (Yes, the guy who wrote the Hobbits and Lord of the Rings - wrote some fascinating observances of this.)., Haven't you already posted this comment? Move on., What the hell do Hobbits and Lord of the Rings have to do wi

In [13]:
# create all possible thread combinations
new_items = []
def create_threads(row):
    for i in range(1, len(row['text']) + 1):
        x = row['text'][:i] # all before
        if not max_previous is None:
            x = x[-max_previous:]
        new = 'xx_thread_start ' + ' '.join([ 'xx_comment_start ' + xx + ' xx_comment_end' for xx in list(x)]) + ' xx_thread_end'
        new_items.append({'text': new, 'sdid': row['sdid']})

In [14]:
for _, row in res.iterrows():
    create_threads(row)

In [15]:
final = pd.DataFrame(new_items)

In [16]:
# final['text'] = final['text'].swifter.apply(lambda x: clean(x, lower=False))

In [17]:
final.groupby('sdid').count()

,text
sdid,
27,7
44,7
80,4
219,5
288,6
311,5
442,7
484,6
523,5


In [18]:
final.shape

(236706, 2)

In [19]:
split_id = 130000
final["sdid"] = pd.to_numeric(final["sdid"])
train = final[final['sdid'] <= split_id][['text']]
val = final[final['sdid'] > split_id][['text']]

In [20]:
train

,text
0,"xx_thread_start xx_comment_start She was 00, not 00. And these prosecuting lawyers act like they have never done anything wrong or questionable in their life. If they look happy, and say they are happy, just leave them alone. Why ruin someone else's life, when there really isn't any good reason to do so? xx_comment_end xx_thread_end"
1,"xx_thread_start xx_comment_start She was 00, not 00. And these prosecuting lawyers act like they have never done anything wrong or questionable in their life. If they look happy, and say they are happy, just leave them alone. Why ruin someone else's life, when there really isn't any good reason to do so? xx_comment_end xx_comment_start Yea, that totally makes it alright for a teacher to bang one of his students. Herp derp. xx_comment_end xx_thread_end"
2,"xx_thread_start xx_comment_start She was 00, not 00. And these prosecuting lawyers act like they have never done anything wrong or questionable in their life. If they look happy, and say they are happy, just leave them alone. Why ruin someone else's life, when there really isn't any good reason to do so? xx_comment_end xx_comment_start Yea, that totally makes it alright for a teacher to bang one of his students. Herp derp. xx_comment_end xx_comment_start Yeah, because #$%$ feelings and human nature, that is all deranged. xx_comment_end xx_thread_end"
3,"xx_thread_start xx_comment_start She was 00, not 00. And these prosecuting lawyers act like they have never done anything wrong or questionable in their life. If they look happy, and say they are happy, just leave them alone. Why ruin someone else's life, when there really isn't any good reason to do so? xx_comment_end xx_comment_start Yea, that totally makes it alright for a teacher to bang one of his students. Herp derp. xx_comment_end xx_comment_start Yeah, because #$%$ feelings and human nature, that is all deranged. xx_comment_end xx_comment_start are you serious no its not ok to bang a student if shes in highschool and your her teacher no matter what her age is xx_comment_end xx_thread_end"
4,"xx_thread_start xx_comment_start She was 00, not 00. And these prosecuting lawyers act like they have never done anything wrong or questionable in their life. If they look happy, and say they are happy, just leave them alone. Why ruin someone else's life, when there really isn't any good reason to do so? xx_comment_end xx_comment_start Yea, that totally makes it alright for a teacher to bang one of his students. Herp derp. xx_comment_end xx_comment_start Yeah, because #$%$ feelings and human nature, that is all deranged. xx_comment_end xx_comment_start are you serious no its not ok to bang a student if shes in highschool and your her teacher no matter what her age is xx_comment_end xx_comment_start Oblivion, you are not the brightest bulb in the room. A) The incident occurred in 0000 which would make the student 00 or 00 at the time of the incident not 00 and the teacher 00 or 00. B) It is and should be against the law for high school teachers to sleep with their students. And C) A..."
5,"xx_thread_start xx_comment_start She was 00, not 00. And these prosecuting lawyers act like they have never done anything wrong or questionable in their life. If they look happy, and say they are happy, just leave them alone. Why ruin someone else's life, when there really isn't any good reason to do so? xx_comment_end xx_comment_start Yea, that totally makes it alright for a teacher to bang one of his students. Herp derp. xx_comment_end xx_comment_start Yeah, because #$%$ feelings and human nature, that is all deranged. xx_comment_end xx_comment_start are you serious no its not ok to bang a student if shes in highschool and your her teacher no matter what her age is xx_comment_end xx_comment_start Oblivion, you are not the brightest bulb in the room. A) The incident occurred in 0000 which would make the student 00 or 00 at the time of the incident not 00 and the teacher 00 or 00. B) It is and should be ag

In [21]:
val

,text
12627,"xx_thread_start xx_comment_start A monster child ivanka would be proud of her monster hitler dad drumpf. That is common sense. Hitler only had his mistress that followed him to their destructive end. This evil family cannot see past their noses to see any difference. Jonathan sticked to Evil Saul leadership to death even though Saul tried to kill David that would later be King of Israel. Her husband believes in an "" eye for an eye ""because he is jewish & drumpf thinks the same way . PALISTINES LIVE IN RUBBLE because of that same mentality so yes, hitler drumpf would destroy America trying to make it great again along with his evil daughter. Chelsea should consider making better friends with stronger God-fearing people. Ivanka would not step in if she was waterboarded. Germans didn't intervene hitlers tortures nor did his mistress. xx_comment_end xx_thread_end"
12628,"xx_thread_start xx_comment_start A monster child ivanka would be proud of her monster hitler dad drumpf. That is common sense. Hitler only had his mistress that followed him to their destructive end. This evil family cannot see past their noses to see any difference. Jonathan sticked to Evil Saul leadership to death even though Saul tried to kill David that would later be King of Israel. Her husband believes in an "" eye for an eye ""because he is jewish & drumpf thinks the same way . PALISTINES LIVE IN RUBBLE because of that same mentality so yes, hitler drumpf would destroy America trying to make it great again along with his evil daughter. Chelsea should consider making better friends with stronger God-fearing people. Ivanka would not step in if she was waterboarded. Germans didn't intervene hitlers tortures nor did his mistress. xx_comment_end xx_comment_start Wow mary, project much? xx_comment_end xx_thread_end"
12629,"xx_thread_start xx_comment_start A monster child ivanka would be proud of her monster hitler dad drumpf. That is common sense. Hitler only had his mistress that followed him to their destructive end. This evil family cannot see past their noses to see any difference. Jonathan sticked to Evil Saul leadership to death even though Saul tried to kill David that would later be King of Israel. Her husband believes in an "" eye for an eye ""because he is jewish & drumpf thinks the same way . PALISTINES LIVE IN RUBBLE because of that same mentality so yes, hitler drumpf would destroy America trying to make it great again along with his evil daughter. Chelsea should consider making better friends with stronger God-fearing people. Ivanka would not step in if she was waterboarded. Germans didn't intervene hitlers tortures nor did his mistress. xx_comment_end xx_comment_start Wow mary, project much? xx_comment_end xx_comment_start What a bunch of garbage, weirdo. xx_comment_end xx_thread_end"
12630,"xx_thread_start xx_comment_start A monster child ivanka would be proud of her monster hitler dad drumpf. That is common sense. Hitler only had his mistress that followed him to their destructive end. This evil family cannot see past their noses to see any difference. Jonathan sticked to Evil Saul leadership to death even though Saul tried to kill David that would later be King of Israel. Her husband believes in an "" eye for an eye ""because he is jewish & drumpf thinks the same way . PALISTINES LIVE IN RUBBLE because of that same mentality so yes, hitler drumpf would destroy America trying to make it great again along with his evil daughter. Chelsea should consider making better friends with stronger God-fearing people. Ivanka would not step in if she was waterboarded. Germans didn't intervene hitlers tortures nor did his mistress. xx_comment_end xx_comment_start Wow mary, project much? xx_comment_end xx_comment_start What a bunch of garbage, weirdo. xx_comment_end xx_comment_start ..."
12631,xx_thread_start xx_comment_start Rumors are spread by the press. Don't ever listen to opinions of the press. The only time I listen is when the candidates 

In [22]:
OUT

PosixPath('~/data/ynacc_proc/replicate/threads_prev6')

In [23]:
! mkdir -p $OUT

In [24]:
train.to_csv(OUT/'train.csv', index=False)

In [25]:
val.to_csv(OUT/'val.csv', index=False)

In [22]:
dff = pd.read_csv('~/data/ynacc_proc/replicate/threads/train_cl.csv')

In [30]:
dff['text_threads']

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              xx_thread_start xx_comment_start Sad to hear such a bad thing. Very dangerous job working on electricity. One questions though, why did they use a picture the Bates house from Psycho, on a Disney story? Or is that what the Paris Haunted Mansion/Phantom Manor looks like? xx_comment_end xx_comment_start Yes..because too many houses in EU look like the original Disney Hunted House so it didn't look scary enough. Bates Motel looks more American and that notion alone scares everyone. xx_comment_end xx_thr

In [28]:
for x in dff['text_threads'].values.tolist():
    print(x.count('xx_comment_end'))

2
3
1
2
3
4
5
1
2
3
4
5
6
7
1
2
3
4
1
2
3
4
5
1
2
3
4
5
6
7
8
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
1
2
1
2
3
4
5
6
7
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
1
2
3
4
5
1
2
3
4
1
2
3
4
5
6
1
2
3
4
5
6
7
8
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
6
7
8
9
1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
1
2
3
4
5
6
1
2
3
4
5
1
2
3
4
5
6
1
2
3
4
1
2
3
4
5
6
7
8
9
10
11
12
13
14
1
2
3
4
5
6
7
1
2
3
4
5
6
7
8
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
6
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
1
2
3
4
5
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
1
2
3
4
5
6
7
1
2
3
4
5
6
7
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
6
7
8
9
1
2
3
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
1
2
3
4
5
6
1
2
3
4
5
6
7
1
2
3
4
5
6
7
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
1
2
3
4
5
6
7
1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
10
11
12
13
1
2
3
4
1
2
3
4
5
1
2
3
1
2
3
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
1
2
3
4
5
6
1
2
3
4
5
1
2
3
4
5

13
14
15
1
2
3
1
2
3
4
5
6
1
2
3
4
5
6
7
8
9
1
2
3
4
5
6
7
1
2
3
4
5
6
1
2
3
4
5
6
7
8
1
2
3
4
5
6
1
2
3
4
5
6
1
2
3
4
5
6
7
8
9
1
2
3
4
5
6
7
8
9
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
5
6
7
8
9
10
11
12
13
1
2
3
4
5
6
7
8
9
1
2
3
4
5
6
1
2
3
4
5
6
7
8
9
1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
1
2
3
4
5
6
7
8
9
10
11
12
13
14
1
2
3
4
5
6
7
1
2
3
4
5
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
1
2
3
4
5
6
1
2
3
1
2
3
1
2
3
4
1
2
3
4
5
6
1
2
3
4
5
6
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
6
1
2
3
4
5
1
2
3
1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
1
2
3
4
5
6
7
1
2
3
4
5
6
7
8
1
2
3
4
5
6
7
1
2
3
4
5
6
7
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
1
2
3
4
5
6
1
2
3
4
5
6
7
1
2
3
1
2
3
4
5
6
1
2
3
1
2
3
4
5
1
2
3
4
5
6
7
8
1
2
3
4
5
1
2
3
4
5
6
7
8
9
1
2
3
4
5
1
2
3
4
5
6
7
8
9
1
2
3
4
5
6
1
2
3
4
5
6
1
2
3
4
5
6
1
2
3
4
5
6
7
1
2
3
4
5
1
2
3
4
1
2
3
4
1
2
3
4
5
1
2
3
4
5
6
7
8
9
10
11
12
13
14
1
2
3
4
5
1
2
1
2
3
4
1
2
3
4
5
1
2
3
1
2
3
4
5
1
2
3
4
5
6
7
8